In [1]:
import pickle
import re
from collections import defaultdict, Counter
from tqdm.notebook import tqdm
import pandas as pd
import random
from IPython.display import display

In [2]:
with open('data_processed.pkl', 'rb') as f:
    data = pickle.load(f)

In [64]:
random.seed('1')
dataset = []

cats = data.keys()
for cat, files in data.items():
    all_lines = [line for lines in files.values() for line in lines]
    sampled = random.sample(all_lines, 20000)
    dataset.extend([(cat, line) for line in sampled])
    
random.shuffle(dataset)

In [76]:
len(dataset), list(cats)

(200000, ['易藏', '医藏', '艺藏', '史藏', '佛藏', '集藏', '诗藏', '子藏', '儒藏', '道藏'])

In [66]:
dataset[:10]

[('道藏', '141原本作‘梁’，据《百子全书》改作‘义’。'),
 ('佛藏', '光本一土木形骸，被汝装饰得金碧辉煌矣。虽可暂悦众目，实则常愧我心也。民廿五'),
 ('医藏', '阳气初衰，阴气初胜，故寒湿之气及体。阴气未盛，未能深入，故取俞以泻阴邪，取合以虚阳邪，阳气始衰，故取于合。'),
 ('艺藏', '西神梅：无锡荣文卿植。三瓣短圆，平边，蒲扇式浅兜捧心⑥，刘海舌，“宀”字肩，干长，色绿。虽称为梅，实则水仙门之无上品也。'),
 ('医藏',
  '道之用也，形化气，气化神，神化虚，虚明而万物所以通也。是以古圣人穷通塞之端，得造化之源。忘形以养气，忘气以养神，忘神以养虚，虚实相通是谓大同。故藏之为元精，用之为万灵，舍之为太乙，放之为太清。是以坎离消长于一身，风云发泄于七窍。真气薰蒸而时无寒暑，其纯阳流注而民无死生，是谓神化之道也。'),
 ('易藏',
  '周衰夜明，常星不见；汉失其德，日晕昼昏；女主摄政，遂使纪纲分析；权臣擅威，乃令至柔震动；景藏飞燕，地裂鸣雉，此则后事而星验也。是乃或前事以告祥，或后攻而示罚，莫不若影随形，如声召音。凶谪时至，谴过无差，休应若臻，福善非谬，居远察迩，天高听卑，圣人之言，信其然矣。'),
 ('易藏',
  '京房曰：“天不雨，庙自濡者，其国大饥，人主当之。”《天镜》曰：“人君宫殿门墙润濡，不出一年，狱失囚。”又曰：“宫殿及垣墙无故汁出，如水濯之状，不出八十日，亡国。”《述异记》曰：“宋临王子项在荆州，永光元年，所住庭柘折，栋裂，并自濡，湿汁滴地。明年，被诛。”京房曰：“天不雨而城自濡者，其国大溃乱，相当之。”'),
 ('集藏',
  '如今再说唐卿自送雯青夫人回南之后，不多几天，就奉了着在总理各国事务衙门行走的谕旨，从此每天要上两处衙门，上头又常叫起儿。高中堂、龚尚书新进军机，遇着军国要事，每要请去商量；回得家来，又总是宾客盈门，大有日不暇给的气象。连素爱摩挲的宋、元精椠，黄、顾校文，也只好似荀束袜材，暂置高阁。在自身上看起来，也算得富贵场中的骄子，政治界里的巨灵了。但是国事日糟一日，战局是愈弄愈僵。从他受事到今，两三个月里，水陆处处失败，关隘节节陷落，反觉得忧心如捣，寝馈不安。这日刚在为国焦劳的时候，门上来报闻韵高闻大人要见。'),
 ('集藏',
  '此君气和骨硬，心肠洁净，眼界亦宽，第学问稍有异同处，

In [67]:
examples_num = len(dataset)
train_num = int(0.8 * examples_num)
dev_num = int(0.1 * examples_num)
test_num = int(0.1 * examples_num)

train_num, dev_num, test_num

(160000, 20000, 20000)

In [68]:
train_set = dataset[:train_num]
dev_set = dataset[train_num:train_num + dev_num]
test_set = dataset[train_num + dev_num:]

In [72]:
import csv

def save_dataset(data, data_type):
    with open(f'text_classification/{data_type}.tsv', 'wt') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(['Label', 'Text'])
        for cat, text in data:
            writer.writerow([cat, text])
            

In [73]:
save_dataset(train_set, 'train')
save_dataset(dev_set, 'dev')
save_dataset(test_set, 'test')

In [74]:
!ls text_classification -lh

total 55M
-rw-rw-r-- 1 yantan yantan 5.4M 4月  30 15:25 dev.tsv
-rw-rw-r-- 1 yantan yantan 5.5M 4月  30 15:25 test.tsv
-rw-rw-r-- 1 yantan yantan  44M 4月  30 15:25 train.tsv


In [10]:
import csv
data_type = 'test'

with open(f'text_classification/{data_type}.tsv', 'rt') as f:
    reader = csv.reader(f, delimiter='\t')
    next(reader)
    dataset = list(reader)
        

In [11]:
from collections import defaultdict, Counter

counter = Counter([l[0] for l in dataset])

In [12]:
counter

Counter({'道藏': 2000,
         '子藏': 2001,
         '易藏': 1914,
         '艺藏': 2035,
         '佛藏': 2012,
         '诗藏': 2016,
         '医藏': 2125,
         '集藏': 1990,
         '儒藏': 1962,
         '史藏': 1945})